# Byte Analysis and Huffman Compression Testing

This notebook demonstrates how to read binary data from different file formats (.txt, .png, .wav) and test the Huffman compression algorithm.

## Setup

First, let's import the necessary modules and set up our environment.

In [1]:
import os
import sys
from PIL import Image

# Add the current directory to the path to import project modules
sys.path.append(os.path.abspath('.'))

from core.file_utils import read_binary_file, display_all_bytes
from core.huffman import huffman_encode, huffman_decode
from utils.bit_utils import bits_to_bytes

# Additional imports for handling different file types
import wave  # For WAV files

## 1. Reading Text Files

Let's start by reading a text file in binary mode to examine its bytes.

In [ ]:
# Read the example text file in binary mode
text_path = os.path.join('examples', 'input_text.txt')

try:
    text_bytes = read_binary_file(text_path)
    print(f"Number of bytes in text file: {len(text_bytes)}")
    print("\nFirst 50 bytes as integers:")
    print(list(text_bytes[:50]))
    print("\nFirst 50 bytes as ASCII:")
    print(text_bytes[:50].decode('ascii'))
except FileNotFoundError:
    print(f"Text file not found at {text_path}")

## 2. Reading PNG Files

Now let's examine a PNG file's binary data, including its header.

In [ ]:
# Create a simple PNG file if it doesn't exist
png_path = os.path.join('examples', 'test.png')

if not os.path.exists(png_path):
    # Create a small colored image
    img = Image.new('RGB', (100, 100), color='red')
    img.save(png_path)
    print(f"Created test PNG file at {png_path}")

# Read the PNG file in binary mode
png_bytes = read_binary_file(png_path)

# Examine the PNG header (first 8 bytes)
png_header = png_bytes[:8]
print("PNG Header (8 bytes):")
print(f"As integers: {list(png_header)}")
print(f"As hex: {png_header.hex()}")

# Expected PNG header: 89 50 4E 47 0D 0A 1A 0A
print("\nVerifying PNG signature...")
is_png = png_header.startswith(b'\x89PNG\r\n\x1a\n')
print(f"Is valid PNG: {is_png}")

print(f"\nTotal file size: {len(png_bytes)} bytes")

## 3. Reading WAV Files

Let's examine a WAV file's binary data, focusing on its 44-byte header.

In [ ]:
# Function to analyze WAV header
def analyze_wav_header(wav_path):
    with wave.open(wav_path, 'rb') as wav_file:
        print(f"Number of channels: {wav_file.getnchannels()}")
        print(f"Sample width: {wav_file.getsampwidth()} bytes")
        print(f"Frame rate: {wav_file.getframerate()} Hz")
        print(f"Number of frames: {wav_file.getnframes()}")
        print(f"Compression type: {wav_file.getcomptype()}")

# Read a WAV file if it exists
wav_path = os.path.join('examples', 'test.wav')

try:
    # Read the WAV file in binary mode
    wav_bytes = read_binary_file(wav_path)
    
    # Examine the WAV header (first 44 bytes)
    wav_header = wav_bytes[:44]
    print("WAV Header (44 bytes):")
    print(f"As integers: {list(wav_header)}")
    print(f"As hex: {wav_header.hex()}")
    
    print("\nWAV File Analysis:")
    analyze_wav_header(wav_path)
    
    print(f"\nTotal file size: {len(wav_bytes)} bytes")
except FileNotFoundError:
    print(f"WAV file not found at {wav_path}")

## 4. Testing Huffman Compression

Let's test our Huffman compression algorithm on a text file.

In [ ]:
# Read the text file content
try:
    with open(text_path, 'r', encoding='utf-8') as f:
        text_content = f.read()
    
    print(f"Original text length: {len(text_content)} characters")
    print(f"Original size: {len(text_content.encode('utf-8'))} bytes")
    
    # Compress using Huffman coding
    encoded_data, codes = huffman_encode(text_content)
    print(f"\nNumber of unique characters: {len(codes)}")
    print("Huffman codes:")
    for char, code in sorted(codes.items()):
        if char.isprintable():
            print(f"'{char}': {code}")
        else:
            print(f"byte {ord(char)}: {code}")
    
    # Convert encoded data to bytes for storage
    encoded_bytes = bits_to_bytes(encoded_data)
    print(f"\nCompressed size: {len(encoded_bytes)} bytes")
    
    # Calculate compression ratio
    compression_ratio = len(text_content.encode('utf-8')) / len(encoded_bytes)
    print(f"Compression ratio: {compression_ratio:.2f}x")
    
    # Test decompression
    decoded_text = huffman_decode(encoded_data, codes)
    print("\nDecompression test:")
    print(f"Decompressed length: {len(decoded_text)} characters")
    print(f"Decompression successful: {decoded_text == text_content}")
except FileNotFoundError:
    print(f"Text file not found at {text_path}")

## 5. Detailed Byte Value Analysis

Let's analyze and display all byte values in different formats for each file.

### Text File Byte Values

In [ ]:
try:
	text_bytes = read_binary_file(text_path)
	display_all_bytes(text_bytes, "Text File")
except FileNotFoundError:
  print(f"Text file not found at {text_path}")

### PNG File Byte Values

In [ ]:
try:
    png_bytes = read_binary_file(png_path)
    display_all_bytes(png_bytes, "PNG File")
except FileNotFoundError:
    print(f"PNG file not found at {png_path}")

### Encoded Data Byte Values

In [ ]:
if 'encoded_bytes' in locals():
    display_all_bytes(encoded_bytes, "Huffman Encoded Data")
else:
    print("No encoded data available. Run the Huffman compression cells first.")

### SARDINAS PATTERSON TEST